In [1]:
import pandas as pd
import os

#!pip install --upgrade pip
#!pip install google
#!pip install google-cloud
#!pip install google-cloud-storage
#!pip install google-cloud-bigquery

In [2]:
from google.cloud import bigquery
import pandas as np
import numpy as np

/Users/titouan/.pyenv/versions/3.10.6/envs/monitor-reactor/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [26]:
GCP_MONITOR_THE_REACTOR = 'monitor-the-reactor'

RELATIVE_PATH = 'data/raw_data'

BQ_FAULTFREE_TRAIN = 'TEP_FaultFree_training'
BQ_FAULTFREE_TEST = 'TEP_FaultFree_testing'
BQ_FAULTY_TRAIN = 'TEP_Faulty_training'
BQ_FAULTY_TEST = 'TEP_Faulty_testing'

COLUMNS_NAMES = [
       'faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2',
       'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11'
]

COLUMNS_TO_KEEP = (COLUMNS_NAMES)
COLUMNS_TO_DROP = tuple(set(COLUMNS_NAMES) - set())


SAMPLE_DIVISION = 10

In [29]:
def get_data(
    project_id=GCP_MONITOR_THE_REACTOR,
    dataset=BQ_FAULTY_TEST,
    col_to_keep=0,
    col_to_drop=0,
    sample_division=SAMPLE_DIVISION,
) :

    if col_to_keep == 0:
        query = f"""
        SELECT {', '.join(col_to_drop)}
        FROM `{project_id}`.`{dataset}`.`csv`
        WHERE
        MOD(sample, {sample_division}) = 0
        ORDER BY faultNumber, simulationRun, sample
        """

        client = bigquery.Client(project=project_id)
        query_job = client.query(query)
        result = query_job.result()
        df = result.to_dataframe()

        return df

    elif col_to_drop == 0:
        query = f"""
        SELECT {', '.join(col_to_keep)}
        FROM `{project_id}`.`{dataset}`.`csv`
        WHERE
        MOD(sample, {sample_division}) = 0
        ORDER BY faultNumber, simulationRun, sample
        """

        client = bigquery.Client(project=project_id)
        query_job = client.query(query)
        result = query_job.result()
        df = result.to_dataframe()

        return df

In [35]:
get_data(col_to_keep=COLUMNS_TO_KEEP)

NotFound: 404 Not found: Dataset monitor-the-reactor:TEP was not found in location US; reason: notFound, message: Not found: Dataset monitor-the-reactor:TEP was not found in location US

Location: US
Job ID: b2c1f1ca-25c7-42fc-ae62-836a04c90341


In [ ]:
print(COLUMNS_TO_KEEP)
print(COLUMNS_TO_DROP)
print(query)


['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11']
('xmeas_41', 'xmeas_14', 'xmeas_34', 'xmeas_2', 'xmeas_15', 'xmv_3', 'xmv_4', 'xmeas_38', 'xmeas_36', 'xmeas_33', 'simulationRun', 'xmeas_5', 'xmeas_30', 'xmeas_32', 'xmeas_19', 'xmv_7', 'xmeas_8', 'xmv_11', 'xmv_9', 'xmeas_22', 'xmeas_35', 'xmeas_1', 'xmeas_25', 'xmeas_28', 'xmeas_40', 'xmv_8', 'sample', 'xmeas_37', 'xmeas_21', 'xmeas_11', 'xmeas_26', 'xmeas_17', 'xmeas